# Описание:
Собрать и подготовить свой собственный датасет, можно в соответствии с темой будущего проекта.
Аугментировать данные техниками из занятия.

## Данные:
Свой датасет

## Задачи:
1) Взять предобученную модель. 
    * Cтандартную предобученную модель из библиотек PyTorch `torchvision.models`
    * Примеры: ResNet18, VGG16, EfficientNet.
2) Применить несколько стратегий аугментации данных 
    * Определение стратегий аугментации
    * Пример Torchvision: `transforms.RandomHorizontalFlip`, `transforms.ColorJitter`.
    * Пример Albumentations: `HorizontalFlip`, `RandomBrightnessContrast`, `Cutout`, `ShiftScaleRotate`.
    * Пример сложных: <i>MixUp</i>, <i>CutMix</i>, <i>Mosaic</i>, <i>Weather</i>, <i>Style Transfer</i>.
3) Собрать результаты успешности стратегий в сводную табличку.
    * Базовая аугментация (поворот + отражение).
    * Цветовая аугментация (яркость, контраст, HSV).
    * Геометрическая аугментация (масштабирование, искажение).
    * Сложная аугментация (Cutout + MixUp).

Большим плюсом будет применение Pytorch Lightning для упорядочивания вашего проекта, но это не обязательное требование.


## Критерий оценки:
1. Есть сводная табличка в конце со сравнением стратегий.
2. У каждой стратегии есть краткое понятное описание, что именно сделано.
3. Есть визуализация собранного датасета.



In [1]:
import os
import json
import cv2
import torch
import pandas as pd
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import FasterRCNN
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [8]:

# Конфигурация
DATA_ROOT = "DataForModel/images"
TRAIN_ANN = "DataForModel/annotations/instances_Train.json"
VAL_ANN = "DataForModel/annotations/instances_Validation.json"
CLASSES = ["background", "defect"]

# Определение путей
IMAGES_ROOT = "DataForModel/images"  # Корневая директория для изображений
ANNOTATIONS_ROOT = "DataForModel/annotations"  # Корневая директория для аннотаций

TRAIN_IMAGES = os.path.join(IMAGES_ROOT, "Train")  # Путь к тренировочным изображениям
TRAIN_ANN = os.path.join(ANNOTATIONS_ROOT, "instances_Train.json")  # Путь к тренировочным аннотациям
VAL_IMAGES = os.path.join(IMAGES_ROOT, "Validation")  # Путь к валидационным изображениям
VAL_ANN = os.path.join(ANNOTATIONS_ROOT, "instances_Validation.json")  # Путь к валидационным аннотациям



DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
# 1. Определение стратегий аугментации
AUGMENTATION_STRATEGIES = {
    'base': A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=15, p=0.5),
        A.Normalize(mean=[0, 0, 0], std=[1/255, 1/255, 1/255]),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'])),
    
    'color': A.Compose([
        A.RandomBrightnessContrast(p=0.5),
        A.CLAHE(p=0.3),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
        A.Normalize(mean=[0, 0, 0], std=[1/255, 1/255, 1/255]),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'])),
    
    'geo': A.Compose([
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.5),
        A.Perspective(scale=(0.05, 0.1), p=0.3),
        A.Normalize(mean=[0, 0, 0], std=[1/255, 1/255, 1/255]),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'])),
    
    'none': A.Compose([
        A.Normalize(mean=[0, 0, 0], std=[1/255, 1/255, 1/255]),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
}

In [ ]:
# 1. Обновленный класс датасета с улучшенной обработкой ошибок
class FabricDefectDataset(Dataset):
    def __init__(self, root, ann_file, transforms=None):
        self.root = root
        self.transforms = transforms
        
        with open(ann_file) as f:
            data = json.load(f)
        
        # Валидация данных при загрузке
        self.images = {}
        self.anns = {}
        valid_ids = []
        
        for img in data['images']:
            if os.path.exists(os.path.join(root, img['file_name'])):
                self.images[img['id']] = img
                valid_ids.append(img['id'])
            else:
                print(f"Missing image: {img['file_name']}")

        for ann in data['annotations']:
            if ann['image_id'] in valid_ids:
                if ann['image_id'] not in self.anns:
                    self.anns[ann['image_id']] = []
                self.anns[ann['image_id']].append(ann)
        
        self.ids = valid_ids
        print(f"Loaded {len(self.ids)} valid images")

    def __getitem__(self, idx):
        try:
            img_id = self.ids[idx]
            img_info = self.images[img_id]
            img_path = os.path.join(self.root, img_info['file_name'])
            
            # Загрузка с проверкой изображения
            img = cv2.imread(img_path)
            if img is None:
                raise FileNotFoundError(f"Failed to read image: {img_path}")
                
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Обработка аннотаций
            boxes = []
            for ann in self.anns.get(img_id, []):
                x, y, w, h = ann['bbox']
                if w <= 0 or h <= 0:
                    continue  # Пропустить некорректные bbox
                boxes.append([x, y, x + w, y + h])
            
            labels = [1] * len(boxes)

            # Применение аугментаций
            if self.transforms:
                transformed = self.transforms(
                    image=img,
                    bboxes=boxes,
                    labels=labels
                )
                img = transformed['image']
                boxes = transformed['bboxes']
                labels = transformed['labels']

            # Конвертация с проверкой пустых данных
            if len(boxes) == 0:
                boxes = torch.zeros((0, 4), dtype=torch.float32)
            else:
                boxes = torch.as_tensor(boxes, dtype=torch.float32)
            
            labels = torch.as_tensor(labels, dtype=torch.int64)

            # Проверка и нормализация изображения
            if img.dim() == 2:
                img = img.unsqueeze(0).repeat(3, 1, 1)
            elif img.shape[0] == 1:  # Grayscale to RGB
                img = img.repeat(3, 1, 1)
                
            img = img.float() / 255.0  # Явная нормализация

            target = {
                'boxes': boxes,
                'labels': labels,
                'image_id': torch.tensor([img_id]),
                'area': (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]) if len(boxes) > 0 else torch.zeros(0),
                'iscrowd': torch.zeros(len(boxes), dtype=torch.int64)
            }

            return img, target
            
        except Exception as e:
            print(f"Error processing image {img_id}: {e}")
            # Возвращаем пустые данные и фильтруем в collate_fn
            return torch.zeros((3, 256, 256)), {'boxes': torch.zeros((0, 4)), 'labels': torch.zeros(0)}

    def __len__(self):
        return len(self.ids)

# 2. Обновленная функция collate_fn с фильтрацией
def collate_fn(batch):
    images = []
    targets = []
    
    for img, target in batch:
        # Фильтрация битых данных
        if img.shape[0] != 3 or target['boxes'].shape[0] < 0:
            continue
            
        images.append(img)
        targets.append(target)
    
    if len(images) == 0:
        # Возвращаем пустой батч
        return [torch.zeros((3, 256, 256))], [{'boxes': torch.zeros((0, 4)), 'labels': torch.zeros(0)}]
    
    return images, targets

In [22]:
# 3. Модель и обучение
def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
        in_features, 
        num_classes=2  # background + defect
    )
    return model.to(DEVICE)

def train_step(model, optimizer, images, targets):
    model.train()
    images = [image.to(DEVICE) for image in images]
    targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

    loss_dict = model(images, targets)
    losses = sum(loss for loss in loss_dict.values())

    optimizer.zero_grad()
    losses.backward()
    optimizer.step()

    return losses.item()

def evaluate(model, dataloader):
    model.eval()
    metric = MeanAveragePrecision()
    
    with torch.no_grad():
        for images, targets in dataloader:
            images = [img.to(DEVICE) for img in images]
            outputs = model(images)
            
            preds = []
            for output in outputs:
                preds.append({
                    'boxes': output['boxes'].cpu(),
                    'scores': output['scores'].cpu(),
                    'labels': output['labels'].cpu()
                })
            
            targets = [{
                'boxes': t['boxes'].cpu(),
                'labels': t['labels'].cpu()
            } for t in targets]
            
            metric.update(preds, targets)
    
    return metric.compute()


def collate_fn(batch):
    images = []
    targets = []
    
    for img, target in batch:
        # Нормализация изображения
        if img.dtype == torch.uint8:
            img = img.float() / 255.0
        
        images.append(img)
        targets.append(target)
    
    return images, targets

In [25]:
results = {}
strategies = ['base', 'color', 'geo', 'none']
    
for strategy in strategies:
    print(f"\n=== Training with {strategy} augmentation ===")
    
    # Датасеты
    train_dataset = FabricDefectDataset(
        root=TRAIN_IMAGES,
        ann_file=TRAIN_ANN,
        transforms=AUGMENTATION_STRATEGIES[strategy]
    )
    
    val_dataset = FabricDefectDataset(
        root=VAL_IMAGES,
        ann_file=VAL_ANN,
        transforms=None  # На валидации аугментации не нужны
    )
    
    # DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=0,  # Отключаем multiprocessing для отладки
        pin_memory=False
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=2,
        collate_fn=collate_fn,
        num_workers=0,
        pin_memory=False
    )
    
    # Инициализация модели
    model = get_model()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    
    # Обучение
    best_map = 0.0
    for epoch in range(10):
        total_loss = 0.0
        for images, targets in train_loader:
            loss = train_step(model, optimizer, images, targets)
            total_loss += loss
        
        # Валидация
        metrics = evaluate(model, val_loader)
        current_map = metrics['map'].item()
        
        print(f"Epoch {epoch+1}/10 | Loss: {total_loss:.2f} | mAP: {current_map:.3f}")
        
        if current_map > best_map:
            best_map = current_map
            torch.save(model.state_dict(), f"best_{strategy}.pth")
    
    results[strategy] = best_map

# Результаты
print("\n=== Final Results ===")
print(pd.DataFrame.from_dict(results, orient='index', columns=['mAP']))


=== Training with base augmentation ===


AttributeError: 'numpy.ndarray' object has no attribute 'dim'

In [18]:
# Тестирование загрузки данных
for images, targets in train_loader:
    print(f"Batch size: {len(images)}")
    print(f"Image shape: {images[0].shape}")  # Должно быть [3, H, W]
    print(f"Boxes shape: {targets[0]['boxes'].shape}")  # [N, 4]
    break

Batch size: 4
Image shape: torch.Size([3, 1280, 960])
Boxes shape: torch.Size([1, 4])
